In [72]:
import os, re
import pandas as pd
import numpy as np

# #显示所有列
# pd.set_option('display.max_columns', None)

# #显示所有行
# pd.set_option('display.max_rows', None)

pd.set_option('max_colwidth',100) 

In [73]:
folders = [
    # r'D:\code\forecast_model\notebook\20250811_base',
    # r'D:\code\forecast_model\notebook\20250811_small_large',
    # r'D:\code\forecast_model\notebook\20250811_extraLarge',
    # r"D:\code\forecast_model\notebook\20250815_base_large",
    r'D:\code\forecast_model\notebook\20250817_base',
    ]

In [74]:
data = pd.DataFrame()
for folder in folders:
    for i in os.listdir(folder):
        if 'muilt_model' == i:continue
        # 20250811_base_P100_DeepLOB_v2_input_indepent	0.558035	2.77h
        # 20250811_base_P100_DeepLOB_v2_overfit
        if 'input_indepent' in i or 'overfit' in i: continue

        if i.endswith('.csv'): continue
        if i.endswith('.xlsx'): continue

        code = i.split('_')[3]
        fold = i.split('_')[4]

        train_folder = os.path.join(folder, i)
        if not os.path.isdir(train_folder): continue

        result_file = os.path.join(train_folder, 'result.csv')
        if not os.path.exists(result_file): continue

        # 修复替换
        _data = open(result_file, 'r').readlines()
        _data[1] = _data[1].replace('(30, 2, 1)', '30@2@1')
        with open(result_file, 'w') as f:
            f.writelines(_data)

        _data = pd.read_csv(result_file)
        # _data['describe.1'] = _data['describe.1'] + "_" + code + f"_fold{fold}"

        # 判断是否有nan
        if _data.isnull().values.any(): 
            print(i)
            continue

        if 'test_class_f1_0' in list(_data):
            _data[f'test_mean_class_f1'] = (_data[f'test_class_f1_0'] + _data[f'test_class_f1_1']) / 2
        elif 'test_final_class_f1_0' in list(_data):
            # 同时采用 best/final
            d2 = _data.copy()
            d2[f'test_mean_class_f1'] = _data[f'test_best_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                d2[f'test_mean_class_f1'] += _data[f'test_best_class_f1_{i}']
            d2[f'test_mean_class_f1'] = d2[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            d2['train_title'] += '_best'

            _data[f'test_mean_class_f1'] = _data[f'test_final_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                _data[f'test_mean_class_f1'] += _data[f'test_final_class_f1_{i}']
            _data[f'test_mean_class_f1'] = _data[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            _data['train_title'] += '_final'

            _data = pd.concat([_data, d2], ignore_index=True)

        # for _type in ['train', 'val', 'test_best', 'test_final', 'test_dummy']:
        #     _data[f'{_type}_mean_class_f1'] = (_data[f'{_type}_class_f1_0'] + _data[f'{_type}_class_f1_1']) / 2

        # for _type in ['train', 'val', 'test_best', 'test_final']:
            # _data[f'{_type}_mean_class_f1_enhanced_pct'] = 100 * (_data[f'{_type}_mean_class_f1'] - _data['test_dummy_mean_class_f1']) / _data['test_dummy_mean_class_f1']

        data = pd.concat([data, _data], ignore_index=True)

try:
    data['mean_test_01_f1'] = (data['test_best_class_f1_0'] + data['test_best_class_f1_1']) / 2
    data['mean_val_01_f1'] = (data['val_class_f1_0'] + data['val_class_f1_1']) / 2
except:
    pass

# 只使用 top 数据集 predict_n100
# data = data.loc[data['train_title'].str.contains('top5')].reset_index(drop=True)
# data = data.loc[data['train_title'].str.contains('predict_n100')].reset_index(drop=True)

data

,train_title,root,alist_upload_folder,amp,epochs,batch_n,batch_size,learning_rate,abs_learning_rate,no_better_stop,...,test_dummy_mean_class_f1,train_mean_class_f1_enhanced_pct,val_mean_class_f1_enhanced_pct,test_best_mean_class_f1_enhanced_pct,test_final_mean_class_f1_enhanced_pct,each_epoch_cost,cost,test_mean_class_f1,mean_test_01_f1,mean_val_01_f1
0,20250817_base_P100_DeepLOB_v2_seed0_IDX0_final,20250817_base_P100_DeepLOB_v2_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.499308,82.242001,27.293609,26.504651,34.597995,0.06h,12.33h,0.670162,0.631648,0.635587
1,20250817_base_P100_DeepLOB_v2_seed0_IDX0_best,20250817_base_P100_DeepLOB_v2_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.499308,82.242001,27.293609,26.504651,34.597995,0.06h,12.33h,0.622507,0.631648,0.635587
2,20250817_base_P100_DeepLOB_v2_seed0_IDX1_final,20250817_base_P100_DeepLOB_v2_seed0_IDX1,train_data,no,200,128,8192,0.0384,0,0,...,0.499429,80.962298,27.064908,27.265301,32.403316,0.08h,15.18h,0.648176,0.635600,0.634599
3,20250817_base_P100_DeepLOB_v2_seed0_IDX1_best,20250817_base_P100_DeepLOB_v2_seed0_IDX1,train_data,no,200,128,8192,0.0384,0,0,...,0.499429,80.962298,27.064908,27.265301,32.403316,0.08h,15.18h,0.624569,0.635600,0.634599
4,20250817_base_P100_DeepLOB_v2_seed0_IDX2_final,20250817_base_P100_DeepLOB_v2_seed0_IDX2,train_data,no,200,128,8192,0.0384,0,0,...,0.499286,79.838999,21.232168,26.443278,27.029498,0.08h,17.00h,0.641886,0.631314,0.605295
5,20250817_base_P100_DeepLOB_v2_seed0_IDX2_best,20250817_base_P100_DeepLOB_v2_seed0_IDX2,train_data,no,200,128,8192,0.0384,0,0,...,0.499286,79.838999,21.232168,26.443278,27.029498,0.08h,17.00h,0.614926,0.631314,0.605295
6,20250817_base_P100_DeepLOB_v2_seed1_IDX3_final,20250817_base_P100_DeepLOB_v2_seed1_IDX3,train_data,no,200,128,8192,0.0384,0,0,...,0.498581,88.000846,31.233659,29.549446,37.349760,0.06h,11.81h,0.684946,0.645908,0.654305
7,20250817_base_P100_DeepLOB_v2_seed1_IDX3_best,20250817_base_P100_DeepLOB_v2_seed1_IDX3,train_data,no,200,128,8192,0.0384,0,0,...,0.498581,88.000846,31.233659,29.549446,37.349760,0.06h,11.81h,0.635443,0.645908,0.654305
8,20250817_base_P100_DeepLOB_v2_seed1_IDX4_final,20250817_base_P100_DeepLOB_v2_seed1_IDX4,train_data,no,200,128,8192,0.0384,0,0,...,0.498905,82.906085,25.906917,26.302783,30.958115,0.07h,14.54h,0.624724,0.630131,0.628156
9,20250817_base_P100_DeepLOB_v2_seed1_IDX4_best,20250817_base_P100_DeepLOB_v2_seed1_IDX4,train_data,no,200,128,8192,0.0384,0,0,...,0.498905,82.906085,25.906917,26.302783,30.958115,0.07h,14.54h,0.620368,0.630131,0.628156


In [75]:
if len(folders) == 1:
    data.to_csv(os.path.join(folders[0], 'result.csv'), index=False)


In [76]:
list(data)

['train_title',
 'root',
 'alist_upload_folder',
 'amp',
 'epochs',
 'batch_n',
 'batch_size',
 'learning_rate',
 'abs_learning_rate',
 'no_better_stop',
 'checkpointing_steps',
 'label_smoothing',
 'weight_decay',
 'classify',
 'y_n',
 'debug',
 'test',
 'seed',
 'need_meta_output',
 'label_train',
 'label_val',
 'label_test_final',
 'model',
 'total_grad_norm_best',
 'total_grad_norm',
 'train_loss_best',
 'train_loss',
 'train_acc_best',
 'train_acc',
 'train_f1_best',
 'train_f1',
 'train_recall_best',
 'train_recall',
 'train_class_acc_0_best',
 'train_class_acc_0',
 'train_class_mcc_0_best',
 'train_class_mcc_0',
 'train_class_f1_0_best',
 'train_class_f1_0',
 'train_class_recall_0_best',
 'train_class_recall_0',
 'train_class_acc_1_best',
 'train_class_acc_1',
 'train_class_mcc_1_best',
 'train_class_mcc_1',
 'train_class_f1_1_best',
 'train_class_f1_1',
 'train_class_recall_1_best',
 'train_class_recall_1',
 'val_loss_best',
 'val_loss',
 'val_acc_best',
 'val_acc',
 'val_f1_be

In [77]:
data[['train_title', 'label_train']]

,train_title,label_train
0,20250817_base_P100_DeepLOB_v2_seed0_IDX0_final,1(810372)@1(810372)
1,20250817_base_P100_DeepLOB_v2_seed0_IDX0_best,1(810372)@1(810372)
2,20250817_base_P100_DeepLOB_v2_seed0_IDX1_final,1(1079480)@1(1079480)
3,20250817_base_P100_DeepLOB_v2_seed0_IDX1_best,1(1079480)@1(1079480)
4,20250817_base_P100_DeepLOB_v2_seed0_IDX2_final,1(1292891)@1(1292891)
5,20250817_base_P100_DeepLOB_v2_seed0_IDX2_best,1(1292891)@1(1292891)
6,20250817_base_P100_DeepLOB_v2_seed1_IDX3_final,1(810372)@1(810372)
7,20250817_base_P100_DeepLOB_v2_seed1_IDX3_best,1(810372)@1(810372)
8,20250817_base_P100_DeepLOB_v2_seed1_IDX4_final,1(1079480)@1(1079480)
9,20250817_base_P100_DeepLOB_v2_seed1_IDX4_best,1(1079480)@1(1079480)


In [78]:
# 多seed平均
key_col_name = ['train_f1']
key_col_name = ['train_f1', 'val_f1']

d = data.loc[:, key_col_name + ['cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'seed' not in i]))

# 只考察 final
_match_bool = data['train_title'].apply(lambda x: 'final' in x)
d = d.loc[_match_bool]

# 根据 idx 区分类别
# 用于未区分title的情况
_cls = d['train_title'].apply(lambda x: [(int(i.replace('IDX', '')) % 3) for i in x.split('_') if 'IDX' in i][0])
_cls = _cls.apply(lambda x: ['nomove30', 'nomove50', 'nomove70'][x])
d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i])) + '_' + _cls

# # title 已经区分
# d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values(key_col_name,ascending=False)


,train_f1,val_f1,cost
train_title,,,
20250817_base_P100_DeepLOB_v2_final_nomove30,0.920442,0.648930,12.07h
20250817_base_P100_DeepLOB_v2_final_nomove50,0.906923,0.638214,14.863999999999999h
20250817_base_P100_DeepLOB_v2_final_nomove70,0.897084,0.620636,16.922h


In [8]:
# 多IDX平均
key_col_name = 'val_f1'
key_col_name = 'label_train'

d = data.loc[:, [key_col_name, 'cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i and 'seed' not in i ]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values([key_col_name],ascending=False)

TypeError: agg function failed [how->mean,dtype->object]